In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import re
from matplotlib.pyplot import figure, show, axes, sci
from matplotlib import cm, colors
from matplotlib.font_manager import FontProperties
from numpy import amin, amax, ravel
import numpy.ma as ma

In [2]:
A = ['Li', 'Na', 'K', 'Rb', 'Cs']
B1 = ['Li', 'Na', 'K', 'Rb', 'Cs', 'Cu','Ag','Au']
#B2 = ['Bi','Sb','In','Au','Ta','Mo','Sc','Y','La','Ce','Pr','Nd','Sm','Eu','Gd','Tb','Dy','Er','Tm','Lu']
#### not _3: Ce, Eu, Gd
B2 = ['Bi','Sb','In','Au','Ta','Mo','Sc','Y','La','Pr','Nd','Sm','Tb','Dy','Er','Tm','Lu']
X = ['Cl']

In [3]:
size_left = len(A)
size_right = len(B1)
size_bottom = len(B2)
eg = np.zeros( (size_right,size_left,size_bottom) )
data_eg = open('/Users/yao/Google Drive/data/2116/dos-Cl/Eg_direct_indirect_hull_LT30meV_all','r')

In [4]:
value_eg = {}
value_eg_direct = {}
lines = data_eg.readlines()
for line in lines:
    match = re.match(r"([a-z]+)([ ]+)([a-z]+)([ ]+)([a-z]+)([ ]+)([a-z]+)([ ]+)([0-9.]+)([ ]+)([0-9.]+)", line, re.I)
    if match:
        items = match.groups()
        value_eg[items[0]+items[2]+items[4]+items[6]]=float(items[8])
        value_eg_direct[items[0]+items[2]+items[4]+items[6]]=float(items[10])

In [5]:
#for i in range(len(value_eg)):
#    if value_eg[i] >= 1.5:
#        value_eg[i] = 1.5

In [6]:
for a in range(size_left):
    for b in range(size_bottom):
        for c in range(size_right):
            compound = A[a]+B1[c]+B2[b]+X[0]
            if compound in value_eg:
                eg[c][a][b] = value_eg[compound]
            else:
                eg[c][a][b] = 1000

In [7]:
masked_eg = ma.masked_where(eg>999, eg)

In [8]:
fig = figure()
#cmap = cm.cool
w = 0.75
h = 0.15
ax = []
images = []
vmin = 1e40
vmax = -1e40
cax = fig.add_axes([0.9, 0.2, 0.02, 0.6])

In [9]:
for i in range(size_right):
    pos = [0.1, 0.7 - i*0.65*h, w, h]
    a = fig.add_axes(pos)
    #plt.yticks(np.arange(size_left), A, size='x-small')
    #plt.xticks(np.arange(size_bottom),[]) 
    a.set_yticks(np.arange(size_left))
    a.set_yticklabels(A, size='x-small')
    a.set_xticks(np.arange(size_bottom))
    if i == size_right-1:
        a.set_xticklabels(B2, size='x-small')
    else:
        a.set_xticklabels([])
                
    dd = ravel(masked_eg[i])
    # Manually find the min and max of all colors for
    # use in setting the color scale.
    vmin = min(vmin, amin(dd))
    vmax = max(vmax, amax(dd))
    images.append(a.imshow(masked_eg[i],interpolation = 'nearest',aspect=0.3))
    ax.append(a)
    
    #a2 = a.twinx()
    #a2.set_ylim(a.get_ylim())
    #a2.set_yticks(np.arange(1))
    #a2.set_yticklabels([B1[i]], size='x-small') ####
    #ax.append(a2)


In [10]:
# Set the first image as the master, with all the others
# observing it for changes in cmap or norm.

class ImageFollower:
    'update image in response to changes in clim or cmap on another image'
    def __init__(self, follower):
        self.follower = follower
    def __call__(self, leader):
        self.follower.set_cmap(leader.get_cmap())
        self.follower.set_clim(leader.get_clim())
norm = colors.Normalize(vmin=vmin, vmax=vmax)
for i, im in enumerate(images):
    im.set_norm(norm)
    if i > 0:
        images[0].callbacksSM.connect('changed', ImageFollower(im))
fig.colorbar(images[0], cax)
axes(ax[0])     # Return the current axes to the first one,
sci(images[0])  # because the current image must be in current axes.
cax.set_ylabel('PBE-Eg(eV)', rotation=270, va='baseline')
show()

AttributeError: 'Figure' object has no attribute 'ylabel'